# Launch Docker Instance

```
docker run --name typedb -d -v typedb-data:/opt/ -p 1729:1729 --platform linux/amd64 vaticle/typedb:latest
```

# Check Connectivity

In [1]:
from typedb.driver import *
client = TypeDB.core_driver("192.168.1.170:1729")
client.close()

# Quickstart
(from https://typedb.com/docs/drivers/python/overview)

In [2]:
DB_NAME = "typedb-tutorial-db"
SERVER_ADDR = "192.168.1.170:1729"

with TypeDB.core_driver(SERVER_ADDR) as driver:
    if driver.databases.contains(DB_NAME):
        driver.databases.get(DB_NAME).delete()
    driver.databases.create(DB_NAME)

    with driver.session(DB_NAME, SessionType.SCHEMA) as session:
        with session.transaction(TransactionType.WRITE) as tx:
            tx.query.define("define person sub entity;")
            tx.query.define("define name sub attribute, value string; person owns name;")
            tx.commit()

    with driver.session(DB_NAME, SessionType.DATA) as session:
        with session.transaction(TransactionType.WRITE) as tx:
            tx.query.insert("insert $p isa person, has name 'Alice';")
            tx.query.insert("insert $p isa person, has name 'Bob';")
            tx.commit()
        with session.transaction(TransactionType.READ) as tx:
            results = tx.query.fetch("match $p isa person; fetch $p: name;")
            for json in results:
                print(json)

{'p': {'name': [{'type': {'value_type': 'string', 'label': 'name', 'root': 'attribute'}, 'value': 'Bob'}], 'type': {'root': 'entity', 'label': 'person'}}}
{'p': {'type': {'label': 'person', 'root': 'entity'}, 'name': [{'value': 'Alice', 'type': {'value_type': 'string', 'root': 'attribute', 'label': 'name'}}]}}


# Test LEDHNTR Client

In [1]:
from pprint import pprint, pformat
from ledhntr import LEDHNTR
from ledhntr.data_classes import(
    Attribute, Entity, Relation, Thing
)

from pkg_resources import resource_stream

led = LEDHNTR()
tdb = led.plugins['typedb_client']
DB_NAME = "typedb-tutorial-db"
SCHEMA = resource_stream('ledhntr', 'schemas/schema.tql').name
ROAD_SCHEMA = resource_stream('ledhntr', 'schemas/road.tql').name

tdb.check_db(DB_NAME)

2024-05-06 14:56:27 [INFO] ledhntr[10972] > core.py > (_reload_all_plugins) [209] >  Loading auto_hunter...
2024-05-06 14:56:27 [INFO] ledhntr[10972] > core.py > (_reload_all_plugins) [218] >  Successfully loaded auto_hunter!
2024-05-06 14:56:27 [INFO] ledhntr[10972] > core.py > (_reload_all_plugins) [209] >  Loading censys...
2024-05-06 14:56:27 [INFO] ledhntr[10972] > core.py > (_reload_all_plugins) [218] >  Successfully loaded censys!
2024-05-06 14:56:27 [INFO] ledhntr[10972] > core.py > (_reload_all_plugins) [209] >  Loading compare_things...
2024-05-06 14:56:27 [INFO] ledhntr[10972] > core.py > (_reload_all_plugins) [218] >  Successfully loaded compare_things!
2024-05-06 14:56:27 [INFO] ledhntr[10972] > core.py > (_reload_all_plugins) [209] >  Loading hyas...
2024-05-06 14:56:27 [INFO] ledhntr[10972] > core.py > (_reload_all_plugins) [218] >  Successfully loaded hyas!
2024-05-06 14:56:27 [INFO] ledhntr[10972] > core.py > (_reload_all_plugins) [209] >  Loading jsonflats_client...
2

True

## MISC

### Try Making Relation without connection

In [2]:
from pprint import pprint, pformat
from ledhntr import LEDHNTR
from ledhntr.data_classes import(
    Attribute, Entity, Relation, Thing
)

from pkg_resources import resource_stream

led = LEDHNTR()
tdb = led.plugins['typedb_client']
DB_NAME = "HUNT-IP-ATTRIBUTES"
SCHEMA = resource_stream('ledhntr', 'schemas/schema.tql').name
ROAD_SCHEMA = resource_stream('ledhntr', 'schemas/road.tql').name

tdb.check_db(DB_NAME)
tdb.db_name=DB_NAME

rel = Relation(label='hunt', has=[Attribute(label='hunt-name', value='TEST')])
tdb.add_relation(rel)

# Surprise! This works because I coded in the empty-ent as a fail-safe...
# But we can't have empty-ents all over the damn place.

2024-05-09 09:37:23 [INFO] ledhntr[41612] > core.py > (__init__) [140] >  Successfully loaded configs!
2024-05-09 09:37:24 [INFO] ledhntr[41612] > typedb_client.py > (add_relation) [400] >  Adding Relation <Relation(label=hunt,hunt-name=TEST,has=2,active_roles=1,total_players=1)


True

## Get all DBs

In [2]:
dbs = tdb.get_all_dbs()
all_dbs = []
for dbo in dbs:
    db = str(dbo)
    if db not in all_dbs:
        all_dbs.append(db)
        print(db)

road
typedb-tutorial-db
DUMMY-DATA-DB


## Search DB for results

In [8]:
from pprint import pprint, pformat
tdb.db_name = DB_NAME
alice = Entity(label='person', has=[Attribute(label='name', value='Alice')])
res = tdb.find_things(alice)
pprint(res)
pprint(res[0].to_dict())

[<Entity(label=person,iid=0x826e80018000000000000001,has=1)]
{'_label': 'person',
 '_ledid': None,
 'abstract': False,
 'has': [{'_label': 'name',
          '_value': 'Alice',
          'abstract': False,
          'iid': '0x836f8001280005416c696365',
          'inferred': False,
          'thingtype': 'attribute'}],
 'iid': '0x826e80018000000000000001',
 'inferred': False,
 'keyattr': '',
 'owns': [],
 'plays': [],
 'relations': [],
 'thingtype': 'entity'}


## Bootstrap ROAD (Root of all data)

In [11]:
def _bootstrap_road(dbc):

    # Start with a fresh RoAD DB
    dbc.create_db()
    dbc.write_tql_file(
        file = ROAD_SCHEMA
    )

    # Create base structure
    things = {'attributes':[], 'entities':[], 'relations':[]}
    base_ents = ['actors', 'archives', 'index', 'scratchpad', 'news', 'active-hunts']
    for be in base_ents:
        things['entities'].append(Entity(label=be))
        road = Relation(label='road', players={'related': things['entities']})
    things['relations'].append(road)
    dbc.bulk_add(things, force=True)

    return True

tdb.db_name = 'road'
_bootstrap_road(tdb)

2024-05-06 11:47:56 [INFO] ledhntr[2988] > typedb_client.py > (write_tql_file) [3667] >  Writing TQL file C:\Users\drive\OneDrive\Documents\GitHub\LED-HNTR\ledhntr\schemas\road.tql...
2024-05-06 11:47:56 [INFO] ledhntr[2988] > typedb_client.py > (write_tql_file) [3689] >  Session is open: True
2024-05-06 11:47:56 [INFO] ledhntr[2988] > typedb_client.py > (write_tql_file) [3690] >  Closing 'apparently' open session <typedb.connection.session._Session object at 0x00000252BD1AE5D0>...
2024-05-06 11:47:56 [INFO] ledhntr[2988] > typedb_client.py > (bulk_add) [655] >  Starting bulk_add process...
2024-05-06 11:47:56 [INFO] ledhntr[2988] > typedb_client.py > (bulk_add) [669] >  Processing 6 entities...
2024-05-06 11:47:56 [INFO] ledhntr[2988] > typedb_client.py > (bulk_add_update) [684] >  Starting bulk_update add with 6 things!
2024-05-06 11:47:56 [INFO] ledhntr[2988] > typedb_client.py > (bulk_check) [898] >  Searching for 0 existing things...
2024-05-06 11:47:56 [INFO] ledhntr[2988] > type

True

# Generate Dummy Data

In [6]:
import random
from datetime import datetime, timedelta
from pprint import pprint, pformat
from time import time

def gen_ips(num, existing_ips=[]):
    if existing_ips :
        print(f"Missing {num} IPs - generating more...")
    for i in range(num):
        ip = ".".join(str(random.randint(0,255)) for _ in range(4))
        if ip not in existing_ips:
            existing_ips.append(ip)
    if len(existing_ips) >= num:
        return existing_ips
    missing = num-len(existing_ips)
    existing_ips = gen_ips(missing, existing_ips)
    return existing_ips

def gen_lorem(words, num_sentences=1):
    sentences = []
    for s in range(num_sentences):
        lorem_ipsum_text = (
            "Lorem ipsum dolor sit amet consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore "
            "magna aliqua Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo "
            "consequat Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur "
            "Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum"
        )
        words_list = lorem_ipsum_text.split()
        output_text = ' '.join(random.choice(words_list) for _ in range(words))
        output_text += "."
        sentences.append(f"{output_text}")
    # sentences = [output_text[i:i+num_sentences] + '.' for i in range(0, len(output_text), num_sentences)]

    return ' '.join(sentences)

def roll_die(sides=6):
    return random.randint(1,sides)

def gen_hunts(num_hunts=1, num_ips=50):
    all_hunts = []
    for nh in range(num_hunts):
        ips = gen_ips(num_ips)
        now = datetime.now()
        yesterday = now - timedelta(days=1)
        tomorrow = now + timedelta(days=1)

        date_discovered = Attribute(label='date-discovered', value=yesterday)
        date_seen_yest = Attribute(label='date-seen', value=yesterday)
        date_seen_today = Attribute(label='date-seen', value=now)
        date_seen_tom = Attribute(label='date-seen', value=tomorrow)
        asn = Attribute(label='note', value='TRASH-PANDA-SERVERS')

        ipents = []

        for ip in ips:
            ipattr = Attribute(label='ip-address', value=ip)
            ipent = Entity(label='ip', has=[ipattr, date_discovered, date_seen_yest, date_seen_today, date_seen_tom])
            roll = roll_die(3)
            if roll==3:
                ipent.has.append(asn)
            else:
                randasn = gen_lorem(3).replace(' ', '-').upper()
                randasnattr = Attribute(label='note', value=randasn)
                ipent.has.append(randasnattr)
            ipents.append(ipent)

        nowint = int(time())
        hunt_name = Attribute(label='hunt-name', value=f'HUNT-{nh}-{nowint}')
        hunt_active = Attribute(label='hunt-active', value=True)
        hunt = Relation(label='hunt', has=[hunt_name, hunt_active], players={'related': ipents})
        # pprint(hunt)
        # pprint(hunt.players['related'][0].to_dict())
        all_hunts.append(hunt)
    return all_hunts


## Add Dummy Data To Database

In [3]:
from ledhntr import LEDHNTR
from time import time
from ledhntr.data_classes import(
    Attribute, Entity, Relation, Thing
)

from pkg_resources import resource_stream

led = LEDHNTR()
tdb = led.plugins['typedb_client']
DB_NAME = "DUMMY-DATA-DB"
tdb.db_name = DB_NAME
SCHEMA = resource_stream('ledhntr', 'schemas/schema.tql').name
nukeit=False

if nukeit:
    tdb.delete_db(DB_NAME)

if not tdb.check_db(DB_NAME):
    tdb.db_name = DB_NAME
    tdb.create_db()
    tdb.write_tql_file(file = SCHEMA)

# tdb.add_relation(hunt) # @ This took 5 seconds to write everything

# @ This was way faster, but I don't think it did it right.
# things = {'attributes': [], 'entities': [], 'relations': [hunt]}
# tdb.bulk_add(things)
# led.logger.setLevel('DEBUG')
all_hunts = gen_hunts(num_hunts=2)
stime = time()
led.logger.info(f"START ADD RELATION {stime}")
tdb.add_relation(all_hunts[0])
etime = time()
led.logger.info(f"END ADD RELATION {etime} - Length: {etime-stime}")
stime = time()
led.logger.info(f"START BULK_ADD {stime}")
things = {'attributes': [], 'entities': [], 'relations': [all_hunts[1]]}
tdb.bulk_add(things)
etime = time()
led.logger.info(f"END BULK_ADD {etime} - Length: {etime-stime}")
# led.logger.setLevel('INFO')

2024-05-06 13:57:24 [INFO] ledhntr[44408] > core.py > (__init__) [140] >  Successfully loaded configs!
2024-05-06 13:57:24 [INFO] ledhntr[44408] > 1008700902.py > (<module>) [32] >  START ADD RELATION 1715018244.4700675


Missing 50 IPs - generating more...


2024-05-06 13:57:25 [INFO] ledhntr[44408] > typedb_client.py > (add_entity) [330] >  Adding Entity <Entity(label=ip,con=0.0,ip-address=75.113.184.217,has=8)
2024-05-06 13:57:25 [INFO] ledhntr[44408] > typedb_client.py > (add_entity) [330] >  Adding Entity <Entity(label=ip,con=0.0,ip-address=249.16.63.197,has=8)
2024-05-06 13:57:25 [INFO] ledhntr[44408] > typedb_client.py > (add_entity) [330] >  Adding Entity <Entity(label=ip,con=0.0,ip-address=216.111.174.59,has=8)
2024-05-06 13:57:25 [INFO] ledhntr[44408] > typedb_client.py > (add_entity) [330] >  Adding Entity <Entity(label=ip,con=0.0,ip-address=63.138.119.231,has=8)
2024-05-06 13:57:25 [INFO] ledhntr[44408] > typedb_client.py > (add_entity) [330] >  Adding Entity <Entity(label=ip,con=0.0,ip-address=84.120.133.36,has=8)
2024-05-06 13:57:25 [INFO] ledhntr[44408] > typedb_client.py > (add_entity) [330] >  Adding Entity <Entity(label=ip,con=0.0,ip-address=135.71.111.108,has=8)
2024-05-06 13:57:25 [INFO] ledhntr[44408] > typedb_client.py

# Generate Shodan Test DB

In [1]:
import copy
import random
from datetime import datetime, timedelta
from importlib.resources import files
from pprint import pprint, pformat
from time import time

from ledhntr import LEDHNTR
from ledhntr.data_classes import(
    Attribute, Entity, Relation, Thing
)

led = LEDHNTR()
tdb = led.plugins['typedb_client']
SCHEMA = str(files('ledhntr').joinpath('schemas/schema.tql'))
nukeit=True

DB_NAME = "SHODAN-ARCANE-DOOR"
tdb.db_name = DB_NAME
# Create new DB
if nukeit:
    tdb.delete_db(DB_NAME)
if not tdb.check_db(DB_NAME):
    tdb.create_db()
    tdb.write_tql_file(file=SCHEMA)

# Create Shodan Hunt
shodan = led.load_plugin('shodan')
print(f"#### ADDING NEW HUNT")
new_hunt = shodan.add_hunt(
    dbc = tdb,
    endpoint = 'hosts_search',
    query = 'hash:-1587139099 ssl:"CN=ocserv VPN, O=ocserv"',
    hunt_active=True,
    hunt_name="arcane_door_cert",
    frequency=12,
    confidence=3.0,
    return_things=True
)
print(f"#### NEW HUNT ADDED: {new_hunt}")

# Run hunts
hunts = shodan.find_active_hunts(tdb, ignore_freq=True)
print(f"#### ACTIVE HUNTS: {hunts}")
hunt_results = shodan.run_hunts(active_hunts=hunts)
print(f"#### HUNT RESULTS: {hunt_results}")

# Add Hunt results to DB
shodan.bulk_add_hunt_results(
    dbc=tdb,
    hunt_results=hunt_results,
)

2024-05-15 16:54:06 [INFO] ledhntr[29012] > core.py > (_reload_all_plugins) [158] >  Loading auto_hunter...
2024-05-15 16:54:06 [INFO] ledhntr[29012] > core.py > (_reload_all_plugins) [167] >  Successfully loaded auto_hunter!
2024-05-15 16:54:06 [INFO] ledhntr[29012] > core.py > (_reload_all_plugins) [158] >  Loading censys...
2024-05-15 16:54:06 [INFO] ledhntr[29012] > core.py > (_reload_all_plugins) [167] >  Successfully loaded censys!
2024-05-15 16:54:06 [INFO] ledhntr[29012] > core.py > (_reload_all_plugins) [158] >  Loading compare_things...
2024-05-15 16:54:06 [INFO] ledhntr[29012] > core.py > (_reload_all_plugins) [167] >  Successfully loaded compare_things!
2024-05-15 16:54:06 [INFO] ledhntr[29012] > core.py > (_reload_all_plugins) [158] >  Loading hyas...
2024-05-15 16:54:06 [INFO] ledhntr[29012] > core.py > (_reload_all_plugins) [167] >  Successfully loaded hyas!
2024-05-15 16:54:06 [INFO] ledhntr[29012] > core.py > (_reload_all_plugins) [158] >  Loading jsonflats_client...
2

#### ADDING NEW HUNT


2024-05-15 16:54:08 [INFO] ledhntr[29012] > typedb_client.py > (add_entity) [330] >  Adding Entity <Entity(label=hunt,con=3.0,hunt-name=shodan-arcane_door_cert,has=10)
2024-05-15 16:54:08 [INFO] ledhntr[29012] > hntr.py > (_find_active_hunts) [740] >  Pulling down players for <Entity(label=hunt,con=3.0,iid=0x826e80068000000000000000,hunt-name=shodan-arcane_door_cert,has=10)...
2024-05-15 16:54:08 [INFO] ledhntr[29012] > hntr.py > (_find_active_hunts) [742] >  Added!
2024-05-15 16:54:08 [INFO] ledhntr[29012] > hntr.py > (run_hunts) [1755] >  self.api_confs: {'hosts_search': <APIConfig(hosts_search), 'host_details': <APIConfig(host_details)}
2024-05-15 16:54:08 [INFO] ledhntr[29012] > hntr.py > (search) [2013] >  Running Shodan search against endpoint hosts_search


#### NEW HUNT ADDED: <Entity(label=hunt,con=3.0,iid=0x826e80068000000000000000,hunt-name=shodan-arcane_door_cert,has=10)
#### ACTIVE HUNTS: {'shodan_hosts_search': [<Entity(label=hunt,con=3.0,iid=0x826e80068000000000000000,hunt-name=shodan-arcane_door_cert,has=10)]}


2024-05-15 16:54:09 [INFO] ledhntr[29012] > hntr.py > (_inc_api_counter) [927] >  Total Shodan API calls:
{'_total': 1, 'hosts_search': 1}
2024-05-15 16:54:09 [INFO] ledhntr[29012] > shodan.py > (parse_hosts_search) [304] >  Parsing /shodan/host/search/ results..
2024-05-15 16:54:09 [INFO] ledhntr[29012] > hntr.py > (bulk_add_hunt_results) [1301] >  Adding Shodan Hunt Results...
2024-05-15 16:54:09 [INFO] ledhntr[29012] > hntr.py > (bulk_add_hunt_results) [1361] >  
	[Shodan] - Running dbc.bulk_add for 
	0 attributes, 
	159 entities, and 
	1 relations...
2024-05-15 16:54:09 [INFO] ledhntr[29012] > typedb_client.py > (bulk_add) [669] >  Processing 159 entities...
2024-05-15 16:54:09 [INFO] ledhntr[29012] > typedb_client.py > (bulk_check) [898] >  Searching for 159 existing things...
2024-05-15 16:54:10 [INFO] ledhntr[29012] > typedb_client.py > (bulk_check) [977] >  thing <Entity(label=ip,con=0,ip-address=154.22.235.17,has=5) not in remote things!
2024-05-15 16:54:10 [INFO] ledhntr[2901

#### HUNT RESULTS: {'shodan_hosts_search': {'shodan-arcane_door_cert': {'hunt': <Entity(label=hunt,con=3.0,iid=0x826e80068000000000000000,hunt-name=shodan-arcane_door_cert,has=331), 'found': {'things': [<Entity(label=ip,con=0,ip-address=154.22.235.17,has=4), <Entity(label=geoloc,con=0,comboid=2320e16471f462be421aae6a2105c11447f8a675bd5fb93c331c7f7459f0c638,has=8), <Entity(label=autonomous-system,con=0,comboid=2d7b8be640cc9e636074dde1b7d8e17c7bd2e4826ecba2e0aaf4ddc612753e5c,has=9), <Entity(label=ip,con=0,ip-address=38.54.56.137,has=4), <Entity(label=geoloc,con=0,comboid=278716b66c17e8f090284c10c408801a5d1392a459b4bc0af9d32e6a69900c27,has=8), <Entity(label=autonomous-system,con=0,comboid=f5a8a036b65317087a1fe48de3c21b870346217ebb39129b2e4dede12fb64fa2,has=9), <Entity(label=ip,con=0,ip-address=146.196.81.186,has=4), <Entity(label=geoloc,con=0,comboid=afcc1b05e55c6394da4c9d5d0b7f3045dcd24a4eb98f0b1d33beb10afd1d11cc,has=7), <Entity(label=autonomous-system,con=0,comboid=4bf72c3fd278059a1341b

2024-05-15 16:54:10 [INFO] ledhntr[29012] > typedb_client.py > (bulk_check) [979] >  First time seeing new thing <Entity(label=autonomous-system,con=0,comboid=2d7b8be640cc9e636074dde1b7d8e17c7bd2e4826ecba2e0aaf4ddc612753e5c,has=10)! {'abstract': False, 'iid': None, 'inferred': False, '_label': 'autonomous-system', 'thingtype': 'entity', 'has': [{'abstract': False, 'iid': None, 'inferred': False, '_label': 'country-code', 'thingtype': 'attribute', '_value': 'US'}, {'abstract': False, 'iid': None, 'inferred': False, '_label': 'note', 'thingtype': 'attribute', '_value': '154.22.235.17'}, {'abstract': False, 'iid': None, 'inferred': False, '_label': 'ledid', 'thingtype': 'attribute', '_value': 'autonomous-system_1715806449634_f6c6bf'}, {'abstract': False, 'iid': None, 'inferred': False, '_label': 'isp', 'thingtype': 'attribute', '_value': 'HONG KONG Megalayer Technology Co.,Limited'}, {'abstract': False, 'iid': None, 'inferred': False, '_label': 'as-name', 'thingtype': 'attribute', '_value

True

In [9]:
all_things = shodan.parse_hosts_search(raw=hunt_results['shodan_hosts_search']['shodan-arcane_door_cert']['found']['raw'])
print(f"{len(all_things)} things")

2024-05-15 16:39:14 [INFO] ledhntr[56860] > shodan.py > (parse_hosts_search) [304] >  Parsing /shodan/host/search/ results..


97 things


In [17]:
match1 = hunt_results['shodan_hosts_search']['shodan-arcane_door_cert']['found']['raw']['matches'][0]
loc1 = match1['location']
locs = []
dka = {
    'city': 'city',
    'region_code': 'province',
    'country_code': 'country-code',
    'country_name': 'country',
    # // 'postal_code': 'postal-code',
}
has = [Attribute(label='note', value='192.168.1.100')]
rel1 = shodan._generate_entity_from_data(
    data=loc1,
    datakey_attrlbls=dka,
    label='geoloc',
    has=has,
    # // players = {'located-in': [ent_ip]},
)
print(rel1)
locs.append(rel1)

match2 = hunt_results['shodan_hosts_search']['shodan-arcane_door_cert']['found']['raw']['matches'][1]
loc2 = match2['location']
dka = {
    'city': 'city',
    'region_code': 'province',
    'country_code': 'country-code',
    'country_name': 'country',
    # // 'postal_code': 'postal-code',
}
has = [Attribute(label='note', value='192.168.1.101')]
rel2 = shodan._generate_entity_from_data(
    data=loc2,
    datakey_attrlbls=dka,
    label='geoloc',
    has=has,
    # // players = {'located-in': [ent_ip]},
)
print(rel2)

print(f"rel2 in locs: {rel2 in locs}")

<Entity(label=geoloc,has=6)
<Entity(label=geoloc,has=6)
rel2 in locs: True


In [23]:
print(rel1.to_dict())
print(rel2.to_dict())
rel1==rel2


{'abstract': False, 'iid': None, 'inferred': False, '_label': 'geoloc', 'thingtype': 'entity', 'has': [{'abstract': False, 'iid': None, 'inferred': False, '_label': 'note', 'thingtype': 'attribute', '_value': '192.168.1.100'}, {'abstract': False, 'iid': None, 'inferred': False, '_label': 'ledid', 'thingtype': 'attribute', '_value': 'geoloc_1715805868579_d54b4e'}, {'abstract': False, 'iid': None, 'inferred': False, '_label': 'city', 'thingtype': 'attribute', '_value': 'San Jose'}, {'abstract': False, 'iid': None, 'inferred': False, '_label': 'province', 'thingtype': 'attribute', '_value': 'CA'}, {'abstract': False, 'iid': None, 'inferred': False, '_label': 'country-code', 'thingtype': 'attribute', '_value': 'US'}, {'abstract': False, 'iid': None, 'inferred': False, '_label': 'country', 'thingtype': 'attribute', '_value': 'United States'}], '_ledid': {'abstract': False, 'iid': None, 'inferred': False, '_label': 'ledid', 'thingtype': 'attribute', '_value': 'geoloc_1715805868579_d54b4e'}, 

True

In [16]:
loc2

{'city': 'Tokyo',
 'region_code': '13',
 'area_code': None,
 'longitude': 139.69171,
 'latitude': 35.6895,
 'country_code': 'JP',
 'country_name': 'Japan'}

In [7]:
hunt_results['shodan_hosts_search']['shodan-arcane_door_cert']['found']['things']

[<Entity(label=ip,con=0,ip-address=154.22.235.17,has=6),
 <Entity(label=geoloc,con=0,comboid=2320e16471f462be421aae6a2105c11447f8a675bd5fb93c331c7f7459f0c638,has=10),
 <Entity(label=autonomous-system,con=0,comboid=2d7b8be640cc9e636074dde1b7d8e17c7bd2e4826ecba2e0aaf4ddc612753e5c,has=11),
 <Entity(label=ip,con=0,ip-address=38.54.56.137,has=6),
 <Entity(label=ip,con=0,ip-address=146.196.81.186,has=6),
 <Entity(label=ip,con=0,ip-address=103.51.140.101,has=6),
 <Entity(label=ip,con=0,ip-address=3.29.102.233,has=7),
 <Entity(label=hostname,con=0,fqdn=ec2-3-29-102-233.me-central-1.compute.amazonaws.com,has=7),
 <Entity(label=domain,con=0,domain-name=amazonaws.com,has=7),
 <Entity(label=ip,con=0,ip-address=3.28.242.0,has=7),
 <Entity(label=hostname,con=0,fqdn=ec2-3-28-242-0.me-central-1.compute.amazonaws.com,has=7),
 <Entity(label=ip,con=0,ip-address=45.63.121.250,has=7),
 <Entity(label=hostname,con=0,fqdn=45.63.121.250.vultrusercontent.com,has=7),
 <Entity(label=domain,con=0,domain-name=vultr

In [5]:
# len(hunt_results['shodan_hosts_search']['shodan-arcane_door_cert']['found']['raw']['matches'])
hunt_results['shodan_hosts_search']['shodan-arcane_door_cert']['found']['raw']['matches'][1]

{'hash': -1587139099,
 'asn': 'AS138915',
 'http': {'status': 200,
  'robots_hash': None,
  'redirects': [],
  'securitytxt': None,
  'title': None,
  'sitemap_hash': None,
  'robots': None,
  'server': None,
  'headers_hash': 1194609015,
  'host': '38.54.56.137',
  'html': '<?xml version="1.0" encoding="UTF-8"?>\n<config-auth client="vpn" type="auth-request">\n<version who="sg">0.1(1)</version>\n<auth id="main">\n<message>Please enter your username.</message>\n<form method="post" action="/auth">\n<input type="text" name="username" label="Username:" />\n</form></auth>\n</config-auth>',
  'location': '/',
  'components': {},
  'securitytxt_hash': None,
  'sitemap': None,
  'html_hash': 262209834},
 'os': None,
 'tags': ['self-signed'],
 'timestamp': datetime.datetime(2024, 5, 15, 13, 18, 41, tzinfo=datetime.timezone.utc),
 'isp': 'Kaopu Cloud HK Limited',
 'transport': 'tcp',
 '_shodan': {'region': 'eu',
  'module': 'https',
  'ptr': True,
  'options': {},
  'id': '0df544db-ddd2-4720-95

## Run Enrichments

In [ ]:
# Get Enrichable things
# Build/run enrichments

# MISC

## Raw TypeDB Queries

In [19]:
from ledhntr.data_classes import Query
q = '''$ip_0012c iid 0x826e8002800000000000012c; get $ip_0012c;'''
q = '''$ip_0 isa ip, has ip-address $ip-address_0; $ip-address_0 = "254.98.204.230"; fetch $ip_0 as IP: ip-address;'''
q = '''$ip isa ip, has ip-address $ip1; $ip1 = '254.98.204.230'; get $ip;'''

MyQuery = Query(qtype='match', string=q)
led.logger.setLevel('DEBUG')
final_answers = tdb.raw_query(MyQuery)
pprint(final_answers)
led.logger.setLevel('INFO')

2024-05-06 15:10:32 [DEBUG] ledhntr[10972] > typedb_client.py > (create_transaction) [1635] >  Creating transaction...
2024-05-06 15:10:32 [DEBUG] ledhntr[10972] > typedb_client.py > (create_transaction) [1671] >  Opened TransactionType.READ with SessionType.DATA!
2024-05-06 15:10:32 [DEBUG] ledhntr[10972] > typedb_client.py > (db_query) [1710] >  Executing match TypeQL Query: 
match
    $ip isa ip,
         has ip-address $ip1;
     $ip1 = '254.98.204.230';
     get $ip;
    


[<Entity(label=ip,con=0.0,iid=0x826e80028000000000000000,ip-address=254.98.204.230,has=8)]


In [5]:
tdb.db_name = "DUMMY-DATA-DB"
ip_attr = Attribute(label='ip-address', value='254.98.204.230')
ipent = Entity(label='ip', has=[ip_attr])
pprint(tdb.get_query_from_thing(ipent))

# tdb.find_things(ipent)

(' $ip_0 isa ip, has ip-address $ip-address_0; $ip-address_0 = '
 '"254.98.204.230"; get $ip_0;')
